In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
X_train = torch.FloatTensor([0., 1., 2.])
X_train = X_train.cuda() 
torch.cuda.get_device_name(0)


In [ ]:
import torch
from torchvision.models import efficientnet_b0
from pytorch_benchmark import benchmark


model = efficientnet_b0()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)

sample = torch.randn(8, 3, 224, 224)  # (B, C, H, W)
sample = sample.cuda()
results = benchmark(model, sample, num_runs=100)
results

In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install numpy

In [ ]:
from dataclasses import dataclass

@dataclass
class Parameters():
    number_features: int = 12
    number_classes: int = 6
    num_folds = 4
    batch_size: int = 32
    epochs: int = 10
    verbosity: int = 1
    step_size: int = 374
    number_folds: int = 2
    output_size: int = 6
    

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import KFold
import torch.nn.functional as F


In [ ]:
# Loading Data
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms

# read the CSV file into a DataFrame
df = pd.read_csv("Alle_Messungen_trimmed.csv")
df.head(1)

In [ ]:
df = df.drop(columns=["id", "user", "id_combined"])
df.drop(['Unnamed: 0'], axis=1, inplace=True)
# get all types of the df
df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].astype('int64')//1e9
le = LabelEncoder()
df["class"] = le.fit_transform(df["class"])
X_train, X_test, Y_train, Y_test = ms.train_test_split(df.values[:, 1:13], df.values[:, 13], test_size=0.2, random_state=42)

In [ ]:
print(X_train[0])

In [ ]:
# Load data and preprocess
# split train dataset into x_train and y_train
x_train = torch.tensor(X_train, dtype=torch.float32)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.long)
y_test = torch.tensor(Y_test, dtype=torch.long)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

print(x_train[0])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
x_train = x_train.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
x_test = x_test.to(device)

In [ ]:
# Something like this as first Model
class Model1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Model2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Something like this as third Model
class Model3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        out, _ = self.rnn(x)
        # out shape: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]  # Take only the last hidden state from the last time step
        # out shape: (batch_size, hidden_size)
        out = self.fc(out)
        # out shape: (batch_size, output_size)
        return out



In [ ]:
# Perform cross-validation
models = [Model1(12, 16, 6), Model2(12, 16, 6), Model3(12, 16, 6), RNNModel(12, 16, 6)]
for model_cuda in models:
    model_cuda.to(device)
num_folds = Parameters.number_folds
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_acc_scores = []
for i, (train, test) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {i+1}')
    train_x, train_y = torch.Tensor(x_train[train]), torch.Tensor(y_train[train])
    test_x, test_y = torch.Tensor(x_train[test]), torch.Tensor(y_train[test])
    for j, model in enumerate(models):
        print(f'Model {j+1}')
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.03)
        for epoch in range(Parameters.epochs):
            model.train()
            optimizer.zero_grad()
            output = model(train_x)
            loss = criterion(output, train_y.long())
            loss.backward()
            optimizer.step()
            model.eval()
            with torch.no_grad():
                output = model(test_x)
                _, predicted = torch.max(output, dim=1)
                total = test_y.size(0)
                correct = (predicted == test_y.long()).sum().item()
                accuracy = correct / total
                print(f'Epoch [{epoch+1}/{Parameters.epochs}] Loss: {loss.item():.4f} Accuracy: {accuracy:.4f}')
                
        fold_acc_scores.append(accuracy)
        print(f'Fold {i+1} Model {j+1} Accuracy: {accuracy:.4f}')

# Select the best model
best_model_idx = np.argmax(fold_acc_scores)
best_model = models[best_model_idx]
print(f'Best Model: Model {best_model_idx+1}')

# Evaluate the best model on test data
best_model.eval()
with torch.no_grad():
    test_x = torch.Tensor(x_test)
    test_y = torch.Tensor(y_test)
    output = best_model(test_x)
    _, predicted = torch.max(output, dim=1)
    total = test_y.size(0)
    correct = (predicted == test_y.long()).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# Define your neural network model
class Model2(nn.Module):
    def __init__(self, num_features, num_classes):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(num_features, 16)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate your model
model = Model2(num_features=12, num_classes=6)
# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10000):
    # Forward pass
    output = model(x_train)
    loss = criterion(output, y_train.long())

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(output, dim=1)
    total = y_train.size(0)
    correct = (predicted == y_train).sum().item()
    accuracy = correct / total

    # Print loss and accuracy for this epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2%}'.format(epoch+1, 10000, loss.item(), accuracy))